### Anvesh Chaturvedi, 20161094
### Rohit Kr. Agarwal, 20161011

In [2]:
import warnings
warnings.filterwarnings('ignore')
import librosa
import sklearn
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
from tqdm import tqdm
import IPython.display as ipd
from six.moves import cPickle as pickle
import shutil
import os

### Zero Crossing Rate

In [5]:
def zero_crossing(x, sr):
    zero_crossings = librosa.zero_crossings(x, pad=False)
    return zero_crossings

### Spectral Centroid

In [6]:
#spectral centroid -- centre of mass -- weighted mean of the frequencies present in the sound
def spectral_centroid(x, sr):
    spectral_centroids = librosa.feature.spectral_centroid(x, sr=sr)[0]
    return spectral_centroids;


### Spectral Rolloff

In [8]:
def spectral_rolloff(x, sr):
    spectral_rolloff = librosa.feature.spectral_rolloff(x, sr=sr)[0]
    return spectral_rolloff
# librosa.display.waveplot(x, sr=sr, alpha=0.4)
# plt.plot(t, normalize(spectral_rolloff), color='r')

### MFCC

In [15]:
def mfcc(x, sr, num):
    mfccs = librosa.feature.mfcc(x, n_mfcc=num, sr=sr)
    return mfccs
# librosa.display.specshow(mfccs, sr=sr, x_axis='time')

In [10]:
def feature_extractor(audio_path):
    x , sr = librosa.load(audio_path, sr=44100)
    return (np.sum(zero_crossing(x, sr)), spectral_centroid(x, sr), mfcc(x, sr, 5), spectral_rolloff(x, sr))

In [12]:
data_path = 'IndianSongsDataset/data'
cnt=0
paths = list()
for language in sorted(os.listdir(data_path)):
    curr_path = os.path.join(data_path, language)
    for _dir in sorted(os.listdir(curr_path)):
        if(_dir == 'normal'): continue
        new_path = os.path.join(curr_path, _dir)
        for song in sorted(os.listdir(new_path)):
            newer_path = os.path.join(new_path, song)
            paths.append(newer_path)
print(len(paths))

627


In [16]:
features = {'mfcc':[],'sc':[],'sroll':[],'zc':[]}
for i, path in enumerate(paths):
    print(i,end='\r')
    f1, f2, f3, f4 = feature_extractor(path)
    features['zc'].append(f1)
    features['sc'].append(f2)
    features['mfcc'].append(f3)
    features['sroll'].append(f4)
for i in range(len(features['zc'])):
    features['zc'] = np.array(features['zc'])
    features['zc'] = np.reshape(features['zc'], (-1, 1))


In [18]:
feat_vec = list()
for i in range(len(paths)):
    feat_vec.append(list())
for feat_typ in features:
    if(feat_typ == 'paths'): continue
    for idx, song_feat in enumerate(features[feat_typ]):
        if(feat_typ == 'mfcc'):
            means = np.mean(np.array(song_feat), axis=1)
            stds = np.std(np.array(song_feat), axis=1)
            feat_vec[idx] = feat_vec[idx] + list(means)
            feat_vec[idx] = feat_vec[idx] + list(stds)
        else:
            feat_vec[idx].append(np.mean(np.array(song_feat)))
            feat_vec[idx].append(np.std(np.array(song_feat)))


In [42]:
features['combined'] = feat_vec
features['paths'] = paths

In [49]:
labels_d = {'mfcc':{},'sc':{},'sroll':{},'zc':{}, 'combined':{}}
for feat_type in features:
    if(feat_type == 'paths'): continue
    feat = np.array(features[feat_type])
    if(feat_type == 'mfcc'):
        mfcc_feat = list()
        for i in range(len(feat)):
            mfcc_feat.append(feat[i].flatten())
        feat = np.array(mfcc_feat)
    for num_clusters in range(3,6):
        clf = sklearn.cluster.KMeans(n_clusters=num_clusters)
        kmeans = clf.fit(feat)
        labels = kmeans.labels_
        labels_d[feat_type][num_clusters] = labels

In [51]:
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)
def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di



In [52]:
## Saving Extracted Features
save_dict(features, 'feat.pkl')

## Saving Labels obtained using clustering
save_dict(labels_d, 'labels.pkl')

In [52]:
loaded_features = load_dict('./feat.pkl')
loaded_labels = load_dict('./labels.pkl')

In [55]:
def play_audio(audio_path):
    ipd.Audio(audio_path)
    
def get_audio_cluster(labels_dict, num_clust = 4, curr_cluster_num = 2, feature_type = 'sc'):
    cl_paths = list()
    curr_labels = labels_dict[feature_type][num_clust]
    for i in range(len(curr_labels)):
        if(curr_labels[i] == curr_cluster_num):
            cl_paths.append(features['paths'][i])
    return cl_paths

In [59]:
num_cl = 5
cl_paths = list()

for i in range(num_cl):
    cl_paths.append(get_audio_cluster(labels_d, num_clust = num_cl, curr_cluster_num = i, feature_type='combined'))

# for feat_typ in features:
#     if(feat_typ == 'paths'): continue
#     print(feat_typ)
#     for num_clust in range(3,6):
#         print("="*10)
#         print(str(num_clust) + " Clusters")
#         cluster_list = labels_d[feat_typ][num_clust]
#         for i in range(num_clust):
#             print(np.sum(cluster_list==i))


In [1]:
import os
from IPython.display import display,Audio,HTML
playlist_html=""
audio_html=""
music_dir = u'Music'
count=0

for file in cl_paths[4]:
    if count==0:
        playlist_html = u'''<li class="active"><a href="{0}">{1}</a>
            </li>\n'''.format(file, file)
        audio_html = u'''<audio id="audio" preload="auto" tabindex="0" controls="" type="audio/mpeg">
            <source type="audio/mp3" src="{}">Sorry, your browser does not support HTML5 audio.
            </audio>'''.format(file)
    else:
        playlist_html +=u'''<li><a href="{0}">{1}</a></li>\n'''.format(file, file)
    count += 1

playlist_html = audio_html + u'''\n<ol id="playlist">\n{}</ol>'''.format(playlist_html)
#print playlist_html


playlist_css = """
<style>
#playlist .active a{color:#CC0000;text-decoration:none;}
#playlist li a:hover{text-decoration:none;}
</style>
"""

playlist_js = """
<script>
var audio;
var playlist;
var tracks;
var current;

init();
function init(){
    current = 0;
    audio = $('audio');
    playlist = $('#playlist');
    tracks = playlist.find('li a');
    len = tracks.length - 1;
    audio[0].volume = .10;
    playlist.find('a').click(function(e){
        e.preventDefault();
        link = $(this);
        current = link.parent().index();
        run(link, audio[0]);
    });
    audio[0].addEventListener('ended',function(e){
        current++;
        if(current == len){
            current = 0;
            link = playlist.find('a')[0];
        }else{
            link = playlist.find('a')[current];    
        }
        run($(link),audio[0]);
    });
}
function run(link, player){
        player.src = link.attr('href');
        par = link.parent();
        par.addClass('active').siblings().removeClass('active');
        audio[0].load();
        audio[0].play();
}
</script>
"""
display(HTML(playlist_html))
display(HTML(playlist_css))
display(HTML(playlist_js))

In [65]:
def save_data(paths_list):
    for i in range(len(paths_list)):
        for file_path in paths_list[i]:
            original = file_path
            target = 'results/' + str(i) + '/' + file_path.split('/')[-3] + '_' + file_path.split('/')[-1]
            shutil.copyfile(original, target)
save_data(cl_paths)